In [1]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 10.9 MB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 11.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:000:00:010:00:01m:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 11.6 MB/s eta 0:00:0031m11.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 KB 11.9 MB/s eta 0:00:0031m11.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 KB 11.9 MB/s eta 0:00:0031m11.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 11.8 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 

In [5]:
import os
os.environ['WANDB_API_KEY'] = '05e022479ff98fa806caf4577b084c4ca42d8a48'# 05e022479ff98fa806caf4577b084c4ca42d8a48   44d441cc99e5a3b8eaa6eff95a6dbd13b53de6a6

In [3]:
!./run_finetune.sh

./run_finetune.sh: line 19: python: command not found


In [4]:
# Inference with Best Checkpoint
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
import os

# Paths
base_model_name = 'Qwen/Qwen3-1.7B'
output_dir = 'output/qwen_finetune'

# Find the best checkpoint (latest checkpoint-* folder)
checkpoints = [d for d in os.listdir(output_dir) if d.startswith('checkpoint-')]
if checkpoints:
    # Sort by step number and get the latest
    checkpoints.sort(key=lambda x: int(x.split('-')[1]))
    adapter_path = os.path.join(output_dir, checkpoints[-1])
else:
    adapter_path = os.path.join(output_dir, 'best_val_loss')

test_file = 'dataset/mimic-iv-note-ext-di-bhc/dataset/test_4000_600_chars_last_100.json'

print(f'Loading adapter from: {adapter_path}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load Base Model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)

# Resize embeddings to match checkpoint (important!)
model.resize_token_embeddings(len(tokenizer))
print(f'Resized embeddings to {len(tokenizer)} tokens')

# Load LoRA Adapter
model = PeftModel.from_pretrained(model, adapter_path)
model.eval()
print('Model loaded successfully!')

# Load a sample from test set
with open(test_file, 'r') as f:
    line = f.readline()
    data = json.loads(line)
    sample_text = data['text'] ##thay thanh text tùy ý
    reference = data['summary']

# Prepare Prompt
instruction = "Summarize for the patient what happened during the hospital stay based on this doctor's note:"
response_prefix = "Summary for the patient:"
prompt = f"{instruction}\n{sample_text}\n{response_prefix} "

# Generate
print('Generating summary...')
inputs = tokenizer(prompt, return_tensors='pt').to(device)
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        max_new_tokens=350,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )

prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Extract only the generated part after the response prefix
if response_prefix in prediction:
    prediction = prediction.split(response_prefix)[-1].strip()

print('=' * 80)
print('REFERENCE SUMMARY:')
print(reference)
print('=' * 80)
print('MODEL PREDICTION:')
print(prediction)
print('=' * 80)


FileNotFoundError: [Errno 2] No such file or directory: 'output/qwen_finetune'

In [ ]:
# Gradio Demo: Baseline vs Fine-tuned Model Comparison
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import gradio as gr
import json
import os

# Paths
base_model_name = 'Qwen/Qwen3-1.7B'
output_dir = 'output/qwen_finetune'

# Find the best checkpoint
checkpoints = [d for d in os.listdir(output_dir) if d.startswith('checkpoint-')]
if checkpoints:
    checkpoints.sort(key=lambda x: int(x.split('-')[1]))
    adapter_path = os.path.join(output_dir, checkpoints[-1])
else:
    adapter_path = os.path.join(output_dir, 'best_val_loss')

print(f'Adapter path: {adapter_path}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load Baseline Model
print('Loading baseline model...')
baseline_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True
)
baseline_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
baseline_model.eval()
print('Baseline model loaded!')

# Load Fine-tuned Model
print('Loading fine-tuned model...')
finetuned_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True
)
finetuned_tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
finetuned_model.resize_token_embeddings(len(finetuned_tokenizer))
finetuned_model = PeftModel.from_pretrained(finetuned_model, adapter_path)
finetuned_model.eval()
print('Fine-tuned model loaded!')

# Prompt template
instruction = "Summarize for the patient what happened during the hospital stay based on this doctor's note:"
response_prefix = "Summary for the patient:"

def generate_summary(clinical_note, model, tokenizer, max_tokens=350):
    prompt = f"{instruction}\n{clinical_note}\n{response_prefix} "
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            max_new_tokens=max_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if response_prefix in prediction:
        prediction = prediction.split(response_prefix)[-1].strip()
    return prediction

def compare_models(clinical_note):
    if not clinical_note.strip():
        return "Please enter a clinical note.", "Please enter a clinical note."
    
    baseline_output = generate_summary(clinical_note, baseline_model, baseline_tokenizer)
    finetuned_output = generate_summary(clinical_note, finetuned_model, finetuned_tokenizer)
    
    return baseline_output, finetuned_output

# Load sample data for examples
test_file = 'dataset/mimic-iv-note-ext-di-bhc/dataset/test_4000_600_chars_last_100.json'
examples = []
with open(test_file, 'r') as f:
    for i, line in enumerate(f):
        if i >= 3:  # Only load 3 examples
            break
        data = json.loads(line)
        examples.append([data['text'][:2000]])  # Truncate for display

# Create Gradio Interface
with gr.Blocks(title="Medical Summarization: Baseline vs Fine-tuned") as demo:
    gr.Markdown("# 🏥 Medical Summarization Comparison")
    gr.Markdown("Compare **Baseline Qwen3-1.7B** vs **Fine-tuned on MIMIC-IV BHC**")
    
    with gr.Row():
        input_text = gr.Textbox(
            label="Clinical Note (Doctor's Brief Hospital Course)",
            placeholder="Enter the clinical note here...",
            lines=10
        )
    
    compare_btn = gr.Button("🔍 Compare Models", variant="primary")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 📝 Baseline Model Output")
            baseline_output = gr.Textbox(label="Baseline (Qwen3-1.7B)", lines=8)
        
        with gr.Column():
            gr.Markdown("### ✨ Fine-tuned Model Output")
            finetuned_output = gr.Textbox(label="Fine-tuned (LoRA)", lines=8)
    
    compare_btn.click(
        fn=compare_models,
        inputs=[input_text],
        outputs=[baseline_output, finetuned_output]
    )
    
    gr.Markdown("### 📋 Example Clinical Notes")
    gr.Examples(
        examples=examples,
        inputs=[input_text]
    )

# Launch
demo.launch(share=True)
